In [16]:
import numpy as np

def generate_s(n):
    return np.random.choice([-1,1], size=n)

def apply_h_on_s(s, h):
    return np.convolve(s,h, mode="same")

def generate_gaussian_noise(n, mean, var):
    return np.random.normal(mean, np.sqrt(var), n)

def autocorrelation_matrix(X, M):
    """
    Calculates the autocorrelation matrix with M lags for a given array X.

    Parameters:
    X (np.array): Input 1D array of data points.
    M (int): Number of lags to use for the autocorrelation matrix.

    Returns:
    np.array: The autocorrelation matrix of shape (M+1, M+1).
    """
    # Flatten the input array in case it's multidimensional
    X = X.flatten()
    
    # Create lagged versions of the array
    lagged_data = np.column_stack([np.roll(X, -lag)[:len(X) - M] for lag in range(M + 1)])
    
    # Compute the autocorrelation matrix
    autocorr_matrix = np.corrcoef(lagged_data, rowvar=False)
    
    return autocorr_matrix

def cross_correlation_vector(X, Y, M):
    """
    Calculates the cross-correlation vector for M lags between two vectors X and Y.

    Parameters:
    X (np.array): First input 1D array of data points.
    Y (np.array): Second input 1D array of data points, should be the same length as X.
    M (int): Number of lags to calculate for the cross-correlation.

    Returns:
    np.array: A 1D array containing the cross-correlation values for each lag up to M.
    """
    # Ensure X and Y are 1D arrays and have the same length
    X = X.flatten()
    Y = Y.flatten()
    assert len(X) == len(Y), "X and Y must be of the same length"
    
    # Mean-center the data for unbiased cross-correlation
    X = X - X.mean()
    Y = Y - Y.mean()
    
    # Calculate cross-correlation for each lag up to M
    cross_corr = np.array([
        np.sum(X[:len(X) - lag] * Y[lag:]) / (len(X) - lag)
        for lag in range(M + 1)
    ])
    
    return cross_corr

def calculate_w_opt(x, d, M):
    R = autocorrelation_matrix(x, M)
    R_inv = np.linalg.inv(R)
    p = cross_correlation_vector(x, d, M)
    return R_inv @ p

def apply_fir_filter(w, x):
    x = x.flatten()
    # Ensure the input is a NumPy array
    x = np.asarray(x)
    w = np.asarray(w)
    
    # Get the order of the filter
    M = len(w)
    # Initialize the output signal with zeros
    output_signal = np.zeros(len(x))

    # Apply the FIR filter using convolution
    for n in range(len(x)):
        for k in range(M):
            if n - k >= 0:  # Check bounds to avoid negative indices
                output_signal[n] += w[k] * x[n - k]

    return output_signal  


n = 1000
s = generate_s(n)
s_filt = apply_h_on_s(s, [0.2, 0.7, 1.3])
noise = generate_gaussian_noise(n, 0, 0.01)

x = s_filt + noise

In [25]:
r = autocorrelation_matrix(x, 1)
r

array([[1.        , 0.44987441],
       [0.44987441, 1.        ]])

In [26]:
p_for_lag = {}
for lag in range(6):
    p_for_lag[str(lag)] = cross_correlation_vector(x, s, lag)

p_for_lag["5"]

array([ 0.68229448,  0.15149668, -0.02452389, -0.02205017, -0.10683839,
       -0.06710915])

In [21]:
R = autocorrelation_matrix(x, 1)
R_inv = np.linalg.inv(R)
p = cross_correlation_vector(x, s, 0)
w = R_inv @ p

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 2)

In [23]:
p

array([0.68229448])